In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator, DateFormatter
import urllib.request
import json
import requests
import urllib
import calendar, datetime
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

In [186]:
# Specify request start and end times, format e.g. '2019-09-17T12:00:00.000Z'
start='2019-12-20T09:00:00.000Z'
end='2020-01-06T09:00:00.000Z'
#start='2019-12-21T16:45:00.000Z'
#end='2019-12-21T16:50:00.000Z'

# Time binning
delta = datetime.timedelta(seconds=1)

# PV list
pvs=['SCR_BTS35:PSEL1_D1596:V_RD', 'SCR_BTS35:PSEL2_D1596:V_RD', 'SCR_BTS35:PSEL1_D1596:I_RD', 'SCR_BTS35:PSEL2_D1596:I_RD', 'SCR_BTS35:PSEL1_D1596:V_CSET', 'SCR_BTS35:PSEL1_D1596:V_CSET_PLC', 'SCR_BTS35:PSEL2_D1596:V_CSET', 'SCR_BTS35:PSEL2_D1596:V_CSET_PLC', 'SCR_BTS35:CCG_D1612:VP_RD_1', 'PSS_SCR:RM_N1301:RDR_RD']
#pvs=['SCR_BTS35:PSEL1_D1596:V_RD', 'SCR_BTS35:PSEL2_D1596:V_RD']

In [187]:
# Create vectorized converter (can take list-like objects as arguments)
dateconv = np.vectorize(dt.datetime.fromtimestamp)

dfs = []
payloads = []
df_dict = {}
for n,pv in enumerate(pvs):
    print(n, pv)
    # Generate url string
    baseurl='http://epicsarchiver.frib.msu.edu/retrieval/data/getData.json?'
    args='from='+urllib.parse.quote_plus(start)+'&to='+urllib.parse.quote_plus(end)+'&donotchunk&pv='
    #print(n,pv)
    
    # Get values from Archive Appliance, asking the archive appliance to do the binning
    r=requests.get(baseurl+args+'mean_'+str(delta.seconds)+'('+urllib.parse.quote_plus(pv)+')')
    payload=r.json()
    payloads.append(payload)  
    
    # Create dataframe
    df = pd.DataFrame(payload[0]['data'])
    # Remove unnecessary columns, and format others
    del df['nanos']
    del df['severity']
    del df['status']
    if 'fields' in df.columns:
        del df['fields']
    df = df.rename(columns={'secs':'time','val':pv})
    df['time'] = pd.to_datetime(df['time'], unit='s')

    dfs.append(df)    

dfs = [df.set_index('time') for df in dfs]

0 SCR_BTS35:PSEL1_D1596:V_RD
1 SCR_BTS35:PSEL2_D1596:V_RD
2 SCR_BTS35:PSEL1_D1596:I_RD
3 SCR_BTS35:PSEL2_D1596:I_RD
4 SCR_BTS35:PSEL1_D1596:V_CSET
5 SCR_BTS35:PSEL1_D1596:V_CSET_PLC
6 SCR_BTS35:PSEL2_D1596:V_CSET
7 SCR_BTS35:PSEL2_D1596:V_CSET_PLC
8 SCR_BTS35:CCG_D1612:VP_RD_1
9 PSS_SCR:RM_N1301:RDR_RD


In [188]:
# Need to check that 1 is positive and 2 is negative
new_df = pd.DataFrame(dfs[0])
new_df["HVPSNeg_V_RD"] = dfs[1]['SCR_BTS35:PSEL2_D1596:V_RD']
new_df["HVPSPos_I_RD"] = dfs[2]['SCR_BTS35:PSEL1_D1596:I_RD']
new_df["HVPSNeg_I_RD"] = dfs[3]['SCR_BTS35:PSEL2_D1596:I_RD'] 
new_df["HVPSPos_V_SET"] = dfs[4]['SCR_BTS35:PSEL1_D1596:V_CSET'] 
new_df["HVPSPos_V_PLC"] = dfs[5]['SCR_BTS35:PSEL1_D1596:V_CSET_PLC'] 
new_df["HVPSNeg_V_SET"] = dfs[6]['SCR_BTS35:PSEL2_D1596:V_CSET'] 
new_df["HVPSNeg_V_PLC"] = dfs[7]['SCR_BTS35:PSEL2_D1596:V_CSET_PLC'] 
new_df["CCG_RD"] = dfs[8]['SCR_BTS35:CCG_D1612:VP_RD_1'] 
new_df["XMonitor_RD"] = dfs[9]['PSS_SCR:RM_N1301:RDR_RD']

new_df.rename(columns = {'SCR_BTS35:PSEL1_D1596:V_RD': "HVPSPos_V_RD"}, inplace = True)

In [190]:
# Save file
new_df.to_csv('WF1_130kV_Dec2020.csv')

In [2]:
# Read file
new_df = pd.read_csv('WF1_130kV_Dec2020.csv')
new_df['time'] = pd.to_datetime(new_df['time'])
#new_df['time'] = new_df['time'].dt.strftime('%Y/%m/%d %H:%M:%S')

In [48]:
# Finding times of interest
#new_df.loc[(new_df['time'] > '2019/12/20 18:48:30')] # Time voltage was dropped
int_df = new_df.loc[(new_df['time'] > '2019-12-20 18:48:30') & # Time voltage was dropped
                    ((new_df['HVPSPos_V_RD'] < 129) | (new_df['HVPSNeg_V_RD'] < 129.8))]
int_df

,time,HVPSPos_V_RD,HVPSNeg_V_RD,HVPSPos_I_RD,HVPSNeg_I_RD,HVPSPos_V_SET,HVPSPos_V_PLC,HVPSNeg_V_SET,HVPSNeg_V_PLC,CCG_RD,XMonitor_RD
119243,2019-12-21 18:07:22,84.085609,122.962517,12.387524,30.092836,130.0,130.0,130.0,130.0,3.800000e-08,0.000005
119244,2019-12-21 18:07:23,84.085609,122.962517,12.387524,30.092836,130.0,130.0,130.0,130.0,3.800000e-08,0.000005
250592,2019-12-23 06:36:31,89.227364,125.509979,29.185673,28.591326,130.0,130.0,130.0,130.0,5.900000e-08,0.000006
250593,2019-12-23 06:36:32,89.227364,125.509979,29.185673,28.591326,130.0,130.0,130.0,130.0,5.900000e-08,0.000006
258767,2019-12-23 08:52:46,105.364418,113.475090,28.997986,27.308784,130.0,130.0,130.0,130.0,6.100000e-08,0.000005
258768,2019-12-23 08:52:47,105.364418,113.475090,28.997986,27.308784,130.0,130.0,130.0,130.0,6.100000e-08,0.000005
258769,2019-12-23 08:52:48,105.364418,113.475090,28.997986,27.308784,130.0,130.0,130.0,130.0,3.700000e-08,0.000005
271535,2019-12-23 12:25:34,128.937820,129.874374,30.311806,2.596423,130.0,130.0,130.0,130.0,4.100000e-08,0.000006
271536,2019-12-23 12:25:35,128.937820,129.874374,30.311806,2.596423,130.0,130.0,130.0,130.0,4.100000e-08,0.000006
271537,2019-12-23 12:25:36,128.937820,129.874374,30.311806,2.596423,130.0,130.0,130.0,130.0,4.100000e-08,0.000006


In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plotWF_time(df, dateStart = 'None', dateEnd = 'None'):
    if dateStart != 'None' and dateEnd != 'None':
        red_df = df.loc[(df['time'] > dateStart) & (df['time'] < dateEnd)]
    else:
        red_df = df

    fig = make_subplots( rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.02 )
    
    fig.add_trace( go.Scatter( x = list(red_df.time), y = list(red_df.HVPSPos_V_RD), 
                               name = 'Pos_V_RD', line = dict(color='firebrick', width=1 ) ), row=1, col=1 )
    fig.add_trace( go.Scatter( x = list(red_df.time), y = list(red_df.HVPSPos_V_PLC), 
                               name = 'Pos_V_PLC', line = dict(color='firebrick', width=1, dash='dash') ), row=1, col=1 )
    fig.add_trace( go.Scatter( x = list(red_df.time), y = list(red_df.HVPSNeg_V_RD), 
                               name = 'Neg_V_RD', line = dict(color='royalblue', width=1 ) ), row=1, col=1 )
    fig.add_trace( go.Scatter( x = list(red_df.time), y = list(red_df.HVPSNeg_V_PLC), 
                               name = 'Neg_V_PLC', line = dict(color='royalblue', width=1, dash='dash') ), row=1, col=1 )
    
    fig.add_trace( go.Scatter( x = list(red_df.time), y = list(red_df.HVPSPos_I_RD), 
                               name = 'Pos_I_RD', line = dict(color='firebrick', width=1 ) ), row=2, col=1 )
    fig.add_trace( go.Scatter( x = list(red_df.time), y = list(red_df.HVPSNeg_I_RD), 
                               name = 'Neg_I_RD', line = dict(color='royalblue', width=1 ) ), row=2, col=1 )
    
    fig.add_trace( go.Scatter( x = list(red_df.time), y = list(red_df.CCG_RD), 
                               name = 'CCG_RD', line = dict(color='green', width=1 ) ), row=3, col=1 )
    
    fig.add_trace( go.Scatter( x = list(red_df.time), y = list(red_df.XMonitor_RD), 
                               name = 'XMonitor_RD', line = dict(color='orange', width=1 ) ), row=4, col=1 )

    fig.update_yaxes( title_text="Voltage [kV]", row=1, col=1)
    fig.update_yaxes( title_text="Current [uA]", row=2, col=1)
    fig.update_yaxes( title_text="Pressure [Torr]", row=3, col=1, type="log")
    fig.update_yaxes( title_text="X-ray monitor [Rem]", row=4, col=1)
    fig.update_layout( title_text = "Wien Filter 1  Red: Positive, Blue: Negative", 
                      width = 1000, height = 800, showlegend=False)
    
    fig.show()
    fig.write_image("Figures/fig1.pdf")

In [8]:
# Range of interest
dateStart = '2019-12-23 00:00:00'
dateEnd = '2019-12-23 23:59:59'
#plotWF_time(new_df, dateStart, dateEnd)

In [15]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plotWF_Voltage(df, dateStart = 'None', dateEnd = 'None'):
    if dateStart != 'None' and dateEnd != 'None':
        red_df = df.loc[ (df['time'] > dateStart) & (df['time'] < dateEnd) ]
                      #  & (df['HVPSPos_V_RD'] > 129.9) & (df['HVPSPos_V_RD'] < 130.2)]
    else:
        red_df = df

    fig = make_subplots( rows=3, cols=1, shared_xaxes=False, vertical_spacing=0.02 )
    
    fig.add_trace( go.Histogram( x = red_df.HVPSPos_V_RD, histnorm='probability', autobinx=False, xbins=dict(start=129.8, end=130.2, size=0.001) ), row=1, col=1 )
    fig.add_trace( go.Histogram( x = red_df.HVPSNeg_V_RD, histnorm='probability', autobinx=False, xbins=dict(start=129.8, end=129.9, size=0.001) ), row=2, col=1 )
    fig.add_trace( go.Histogram( x = red_df.HVPSPos_V_RD + red_df.HVPSNeg_V_RD, histnorm='probability', 
                                 xbins=dict(start=259.8, end=260.05, size=0.001)), row=3, col=1 )

    fig.update_yaxes( title_text="Pos. voltage fraction", row=1, col=1) #type="log",
    fig.update_yaxes( title_text="Neg. voltage fraction", row=2, col=1)
    fig.update_yaxes( title_text="Sum voltage fraction", row=3, col=1)
    fig.update_xaxes( title_text="Voltage [kV]", row=3, col=1)
    fig.update_layout( title_text = "Wien Filter 1", width = 1000, height = 800, showlegend=False)
    fig.write_image("Figures/fig1.pdf")
    fig.show()

# Range of interest
dateStart = '2019-12-20 18:48:30'
dateEnd = '2020-01-06 23:59:59'
#plotWF_Voltage(new_df, dateStart, dateEnd)

In [11]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plotWF_Voltage2D(df, dateStart = 'None', dateEnd = 'None'):
    if dateStart != 'None' and dateEnd != 'None':
        red_df = df.loc[ (df['time'] > dateStart) & (df['time'] < dateEnd) ]
                      #  & (df['HVPSPos_V_RD'] > 129.9) & (df['HVPSPos_V_RD'] < 130.2)]
    else:
        red_df = df

    fig = go.Figure( go.Histogram2d( x = red_df.HVPSPos_V_RD, y = red_df.HVPSNeg_V_RD, histnorm='probability', 
                                    autobinx=False, xbins=dict(start=129.9, end=130.25, size=0.01),
                                    autobiny=False, ybins=dict(start=129.75, end=129.95, size=0.01), 
                                    colorscale=[[0, 'rgb(12,51,131)'], [0.25, 'rgb(10,136,186)'], [0.5, 'rgb(242,211,56)'], [0.75, 'rgb(242,143,56)'], [1, 'rgb(217,30,30)']]
                                   ) ) 
    fig.update_xaxes( title_text="Pos. voltage [kV]" )
    fig.update_yaxes( title_text="Neg. voltage [kV]" )
    fig.update_layout( title_text = "Wien Filter 1", width = 1000, height = 800, showlegend=False)
    fig.write_image("Figures/fig1.pdf")
    fig.show()

# Range of interest
dateStart = '2019-12-20 18:48:30'
dateEnd = '2020-01-06 23:59:59'
plotWF_Voltage2D(new_df, dateStart, dateEnd)